In [7]:
import os
import glob
import numpy as np
import pyvips
from tqdm import tqdm
import re
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

In [8]:
!rm /BrainSeg/ground_truth_masks/.ipynb_checkpoints -rf

In [9]:
TRUTH_DIR = '/BrainSeg/ground_truth_masks/'
BASE_DIR = '/BrainSeg/'
RES_DIR = '/BrainSeg/Classify_Results/ResNet_Aug_new/GMBData/'
TEST_DIR = '/BrainSeg/Classify_Results/CRF_0913/GMBData/'
SAVE_DATA_DIR = BASE_DIR + 'Classify_Results/CRF_0914/GMBData/'
if not os.path.exists(SAVE_DATA_DIR):
    os.makedirs(SAVE_DATA_DIR)
print(SAVE_DATA_DIR)
SAVE_IMG_DIR = BASE_DIR + 'Classify_Results/CRF_0914/GMBMasks/'
if not os.path.exists(SAVE_IMG_DIR):
    os.makedirs(SAVE_IMG_DIR)
print(SAVE_IMG_DIR)

/BrainSeg/Classify_Results/CRF_0914/GMBData/
/BrainSeg/Classify_Results/CRF_0914/GMBMasks/


In [10]:
test_img = sorted(os.listdir(TEST_DIR))
image_names = [imagename.split('.')[0] for imagename in test_img]
image_names = image_names
image_names             

['NA4553-02_AB',
 'NA4626-02_AB',
 'NA4672-02_AB',
 'NA4675-02_AB',
 'NA4691-02_AB',
 'NA4695-02_AB',
 'NA4907-02_AB17-24',
 'NA4945-02_AB17-24',
 'NA5015-02_AB17-24',
 'NA5029-02_AB17-24']

In [11]:
import numpy as np
np.set_printoptions(threshold=np.inf)

In [12]:
for img in image_names:
    print(img)
    test_npy = np.load(TEST_DIR + img + '.npy')
    res_npy = np.load(RES_DIR + img + '.npy')
    mtx = res_npy
    w, h = test_npy.shape
    mtx[0:w, 0:h] = test_npy
    np.save(SAVE_DATA_DIR + img + '.npy', mtx)
    nums = mtx
    nums = np.repeat(nums[:, :, np.newaxis], 3, axis=2)
    #print(nums.shape, nums.dtype)

    # nums[:,:,0] = RED, nums[:,:,1] = Green, nums[:,:,2] = Blue
    idx_1 = np.where(nums[:,:,0] == 2)  # Index of label 1 (WM)
    idx_2 = np.where(nums[:,:,0] == 1)  # Index of label 2 (GM)

    # For label 0, leave as black color
    # For label 1, set to cyan color: R0G255B255
    nums[:,:,0].flat[np.ravel_multi_index(idx_1, nums[:,:,0].shape)] = 0
    nums[:,:,1].flat[np.ravel_multi_index(idx_1, nums[:,:,1].shape)] = 255
    nums[:,:,2].flat[np.ravel_multi_index(idx_1, nums[:,:,2].shape)] = 255
    # For label 2, set to yellow color: R255G255B0
    nums[:,:,0].flat[np.ravel_multi_index(idx_2, nums[:,:,0].shape)] = 255
    nums[:,:,1].flat[np.ravel_multi_index(idx_2, nums[:,:,1].shape)] = 255
    nums[:,:,2].flat[np.ravel_multi_index(idx_2, nums[:,:,2].shape)] = 0

    save_img = Image.fromarray(nums, 'RGB')
    save_img.save(SAVE_IMG_DIR + img + '.png')


NA4553-02_AB
NA4626-02_AB
NA4672-02_AB


FileNotFoundError: [Errno 2] No such file or directory: '/BrainSeg/Classify_Results/ResNet_Aug_new/GMBData/NA4672-02_AB.npy'